In [11]:
## No elimine esta celda, ya que permite actualizar los archivos .py
## sin tener que reiniciar el Kernel de Jupyter.
%load_ext autoreload
%autoreload 2
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.datasets import ImageFolder
import numpy as np
import torch
from train import find_best_model, run_experiment
from transforms import Transforms
from utils import  set_seed, count_instances_per_class
import os
import warnings


warnings.filterwarnings("ignore", message = "Palette images with Transparency expressed in bytes should be converted to RGBA images")
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

set_seed()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Para ver todos los resultados véase:

{https://api.wandb.ai/links/benjaminrromo-universidad-adolfo-ib-ez/py96u9rv}

In [12]:
train_transforms = A.Compose(
    [A.Resize(256, 256), A.ToFloat(), ToTensorV2()]
)
validation_transforms = A.Compose(
    [A.Resize(256, 256), A.ToFloat(), ToTensorV2()]
)


train_data = ImageFolder(
    "house_plant_species/train",
    transform = Transforms(train_transforms),
)
validation_data = ImageFolder(
    "house_plant_species/validation",
    transform = Transforms(validation_transforms),
)



print(f"Elementos en Entrenamiento: {len(train_data)}")
print(f"Elementos en Validación: {len(validation_data)}")
print(f"There are {len(train_data.classes)} classes!")
#

# Llamar a las funciones
class_counts = count_instances_per_class(train_data)

# Calcular estadísticas
class_instance_counts = np.array(list(class_counts.values()))
mean_instances = np.mean(class_instance_counts)
std_instances = np.std(class_instance_counts)
min_instances = np.min(class_instance_counts)
max_instances = np.max(class_instance_counts)

# # Mostrar resultados
# print("\nInstancias por clase:\n")
# for class_name, count in class_counts.items():
#     print(f"Clase {class_name}: {count} imágenes\n===========================================")

print("\nEstadísticas de las instancias por clase:\n")
print(f"Promedio de instancias:\n============================== {mean_instances:.2f}")
print(f"Desviación estándar:\n============================== {std_instances:.2f}")
print(f"Cantidad mínima de instancias:\n============================== {min_instances}")
print(f"Cantidad máxima de instancias:\n============================== {max_instances}")

Elementos en Entrenamiento: 10835
Elementos en Validación: 2736
There are 47 classes!

Estadísticas de las instancias por clase:

Promedio de instancias:
============================== 230.53
Desviación estándar:
============================== 77.46
Cantidad mínima de instancias:
============================== 49
Cantidad máxima de instancias:
============================== 432


In [13]:
class_counts

defaultdict(int,
            {'African Violet (Saintpaulia ionantha)': 269,
             'Aloe Vera': 200,
             'Anthurium (Anthurium andraeanum)': 326,
             'Areca Palm (Dypsis lutescens)': 151,
             'Asparagus Fern (Asparagus setaceus)': 135,
             'Begonia (Begonia spp.)': 157,
             'Bird of Paradise (Strelitzia reginae)': 144,
             'Birds Nest Fern (Asplenium nidus)': 232,
             'Boston Fern (Nephrolepis exaltata)': 245,
             'Calathea': 188,
             'Cast Iron Plant (Aspidistra elatior)': 212,
             'Chinese Money Plant (Pilea peperomioides)': 305,
             'Chinese evergreen (Aglaonema)': 411,
             'Christmas Cactus (Schlumbergera bridgesii)': 249,
             'Chrysanthemum': 132,
             'Ctenanthe': 176,
             'Daffodils (Narcissus spp.)': 292,
             'Dracaena': 161,
             'Dumb Cane (Dieffenbachia spp.)': 432,
             'Elephant Ear (Alocasia spp.)': 265,
     

In [14]:
count_instances_per_class(validation_data)

defaultdict(int,
            {'African Violet (Saintpaulia ionantha)': 68,
             'Aloe Vera': 51,
             'Anthurium (Anthurium andraeanum)': 81,
             'Areca Palm (Dypsis lutescens)': 38,
             'Asparagus Fern (Asparagus setaceus)': 34,
             'Begonia (Begonia spp.)': 41,
             'Bird of Paradise (Strelitzia reginae)': 36,
             'Birds Nest Fern (Asplenium nidus)': 58,
             'Boston Fern (Nephrolepis exaltata)': 61,
             'Calathea': 48,
             'Cast Iron Plant (Aspidistra elatior)': 54,
             'Chinese Money Plant (Pilea peperomioides)': 77,
             'Chinese evergreen (Aglaonema)': 103,
             'Christmas Cactus (Schlumbergera bridgesii)': 63,
             'Chrysanthemum': 34,
             'Ctenanthe': 44,
             'Daffodils (Narcissus spp.)': 74,
             'Dracaena': 41,
             'Dumb Cane (Dieffenbachia spp.)': 109,
             'Elephant Ear (Alocasia spp.)': 67,
             'English I

### Best architecture is **efficientnet_b5.sw_in12k** obtained with:

###### "color_jitter", "horizontal_flip", "brightness_contrast", "blur"
###### Trivial fc_layers => Lazylinear(512) => Linear(512, num_clases)
###### Resize to 256 x 256 images
###### ToFloat() and not Normalize

### **New Change**
####         for layer in [self.backbone.conv_head, self.backbone.bn2, self.backbone.global_pool]:
####            for param in layer.parameters():
####                param.requires_grad = True
####  416 x 416 input images resizing


### *New Color_Jitter*:

#### Changed color_jitter to : "color_jitter": A.ColorJitter(brightness = 0.4, contrast = 0.4, saturation = 0.3, hue = 0.2, p = 0.7)
#### from "color_jitter": A.ColorJitter(brightness = 0.2, contrast = 0.2, saturation = 0.2, hue = 0.1, p = 0.5),
#### No center crop, added random_rotate in 15 degrees

In [15]:
run_experiment(
    selected_transforms = ["color_jitter", "horizontal_flip", "brightness_contrast", "noise", "blur", "random_rotate"],
    model_name = "efficientnet_b5.sw_in12k",
    base_name = "",
    extra_info = "bn_drp_unfreeze_416_cj_cp_b88_ff", 
    model_type = "efficientnet_b5.sw_in12k",
    num_classes = 47,
    training_params = {"learning_rate": 0.0003, "batch_size": 88, "num_epochs": 50},
    frozen = True,
    result_file = "results/efficient_net.json",
    normalize = False,
    use_float = True,
    resize_size = 416  # Aquí defines dinámicamente el tamaño de `resize`
)

Running experiment: efficientnet_b5.sw_in12k__bn_drp_unfreeze_416_cj_cp_b88_ff
Using device: cuda


Initial max val_accuracy: 0.9609



  0%|          | 0/50 [00:06<?, ?it/s]


Error durante el experimento: CUDA out of memory. Tried to allocate 698.00 MiB. GPU 0 has a total capacity of 5.70 GiB of which 151.00 MiB is free. Process 9927 has 3.53 GiB memory in use. Including non-PyTorch memory, this process has 1.99 GiB memory in use. Of the allocated memory 1.84 GiB is allocated by PyTorch, and 9.37 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Experimento terminado para efficientnet_b5.sw_in12k__bn_drp_unfreeze_416_cj_cp_b88_ff


In [16]:
file_path = "results/efficient_net.json"
criteria = ["max_val_acc", "best_val_loss"]
 
for crit in criteria:
    print(f"El mejor modelo según {crit} es {find_best_model(file_path, crit)["architecture"]} con {crit} de {find_best_model(file_path, crit)[f"{crit}"]}")

El mejor modelo según max_val_acc es efficientnet_b5.sw_in12k__bn_drp_unfreeze_416_cj_cp_b88_ff con max_val_acc de 0.9608917832374573
El mejor modelo según best_val_loss es efficientnet_b5.sw_in12k__bn_drp_unfreeze_416_cj_cp_b88_ff con best_val_loss de 0.14713793000555597
